In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.utils import Bunch

In [2]:
# Define the chunk size for reading the CSV file
chunksize = 100000  # Adjust this value based on your requirements

# Initialize an empty list to store filtered chunks
filtered_chunks = []

# Read the CSV file in chunks based on the defined chunk size
for chunk in pd.read_csv("/home/athyrson/Code/Data/Raw Data/Lending Club/accepted_2007_to_2018Q4.csv", chunksize=chunksize):
    # Filter the current chunk based on the criteria
    filtered_chunk = chunk[chunk['issue_d'].str.contains("2010", na=False)]
    # filtered_chunk = filtered_chunk[~filtered_chunk['issue_d'].str.contains("Oct-2013|Nov-2013|Dec-2013", na=False)]
    # Append the filtered chunk to the list
    filtered_chunks.append(filtered_chunk)

# Concatenate all filtered chunks into a single DataFrame
df = pd.concat(filtered_chunks)


/tmp/ipykernel_5250/2038679434.py:8: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv("/home/athyrson/Code/Data/Raw Data/Lending Club/accepted_2007_to_2018Q4.csv", chunksize=chunksize):
/tmp/ipykernel_5250/2038679434.py:8: DtypeWarning: Columns (19,59) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv("/home/athyrson/Code/Data/Raw Data/Lending Club/accepted_2007_to_2018Q4.csv", chunksize=chunksize):
/tmp/ipykernel_5250/2038679434.py:8: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv("/home/athyrson/Code/Data/Raw Data/Lending Club/accepted_2007_to_2018Q4.csv", chunksize=chunksize):
/tmp/ipykernel_5250/2038679434.py:8: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv("/home/athyrson/Code/Data/Raw Data/Le

In [5]:
df_1 = df.sample(frac=1, random_state=42)
df_1.shape

(12537, 151)

In [3]:
import re

def preprocess_data(df):
    # Calculate risk score
    df["risk_score"] = df.loc[:, ["last_fico_range_high", "last_fico_range_low"]].mean(axis=1)
    
    # Create target variable
    df["target"] = np.where((df.loan_status == 'Current') |
                            (df.loan_status == 'Fully Paid') |
                            (df.loan_status == "Issued") |
                            (df.loan_status == 'Does not meet the credit policy. Status:Fully Paid'), 0, 1)
    
    # Drop unnecessary columns
    for c in ['last_fico_range_high', 'last_fico_range_low', 'loan_status']:
        df = df.drop(c, axis=1, errors='ignore')
    
    def fix_term_column(df):
        """
        Fix the dtype of the 'term' column by extracting the numeric part and converting it to an integer.
        Handles cases where the column contains non-string values (e.g., NaN or float).
        
        Parameters:
            df (pd.DataFrame): The DataFrame containing the 'term' column.
        
        Returns:
            pd.DataFrame: The DataFrame with the 'term' column converted to numeric.
        """
        try:
            # Extract numeric part from the 'term' column and convert to integer
            df['term'] = df['term'].apply(lambda x: int(re.search(r'\d+', str(x)).group()) if pd.notna(x) else x)
        except AttributeError as e:
            # Handle cases where the regex search fails (e.g., no match found)
            print("Error: Unable to extract numeric value from 'term' column.")
            print(f"Details: {e}")
        except Exception as e:
            # Handle any other unexpected errors
            print("An unexpected error occurred while fixing the 'term' column.")
            print(f"Details: {e}")
        
        return df
    
    def fix_emp_length_column(df):
        """
        Fix the dtype of the 'emp_length' column by converting it to numeric.
        Handles cases where the column contains non-string values (e.g., NaN or float).
        
        Parameters:
            df (pd.DataFrame): The DataFrame containing the 'emp_length' column.
        
        Returns:
            pd.DataFrame: The DataFrame with the 'emp_length' column converted to numeric.
        """
        try:
            # Map '< 1 year' to '0' and extract numeric part from the 'emp_length' column
            df['emp_length'] = df['emp_length'].map(lambda x: "0" if x == '< 1 year' else x)
            df['emp_length'] = df['emp_length'].map(lambda x: int(re.search(r'\d+', str(x)).group()), na_action='ignore')
        except Exception as e:
            # Handle any unexpected errors
            print("Error: Unable to convert 'emp_length' column to numeric.")
            print(f"Details: {e}")
        
        return df
    
    df = fix_term_column(df)
    df = fix_emp_length_column(df)

    return df

In [6]:
df_2 = preprocess_data(df_1)
df_2

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,risk_score,target
1643125,498726,NaN,5000.0,5000.0,5000.000000,36,11.36,164.56,B,B5,...,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,732.0,0
1636584,597920,NaN,10000.0,10000.0,9861.357548,36,7.88,312.82,A,A5,...,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,682.0,0
1639847,543951,NaN,8700.0,8700.0,8700.000000,36,7.51,270.67,A,A4,...,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,787.0,0
1643531,492732,NaN,4000.0,4000.0,4000.000000,36,7.88,125.13,A,A5,...,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,712.0,0
1640954,528880,NaN,10000.0,10000.0,9362.071323,36,7.14,309.42,A,A3,...,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,822.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1652094,562187,NaN,13800.0,13800.0,13800.000000,36,16.32,487.36,D,D5,...,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,602.0,0
1638791,562457,NaN,8000.0,8000.0,8000.000000,36,7.14,247.53,A,A3,...,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,812.0,0
1638990,557251,NaN,5000.0,5000.0,4500.000000,36,14.84,172.94,D,D1,...,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,707.0,0
1634460,624384,NaN,10000.0,10000.0,9950.000000,60,13.35,229.33,C,C4,...,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,702.0,0


In [18]:
from sklearn.datasets import make_classification
from sklearn.feature_selection import mutual_info_classif
X, y = make_classification(
    n_samples=100, n_features=10, n_informative=3, n_clusters_per_class=1,
    shuffle=False, random_state=4
)
mutual_info_classif(X, y)

array([0.16021196, 0.39814807, 0.38840337, 0.14476594, 0.16593686,
       0.01075193, 0.01459115, 0.01923556, 0.04785472, 0.        ])

In [25]:
drop_list = [
    'issue_d', 'url', 'collection_recovery_fee', 'last_pymnt_amnt', 'last_pymnt_d', 'pymnt_plan', 'recoveries', 
    'total_pymnt', 'total_pymnt_inv', 'total_rec_int', 'total_rec_late_fee', 'total_rec_prncp', 'hardship_flag', 
    'hardship_type', 'hardship_reason', 'hardship_status', 'deferral_term', 'hardship_amount', 'hardship_start_date', 
    'hardship_end_date', 'payment_plan_start_date', 'hardship_length', 'hardship_dpd', 'hardship_loan_status', 
    'orig_projected_additional_accrued_interest', 'hardship_payoff_balance_amount', 'hardship_last_payment_amount', 
    'debt_settlement_flag', 'debt_settlement_flag_date', 'settlement_status', 'settlement_date', 'settlement_amount', 
    'settlement_percentage', 'settlement_term', 'id', 'emp_title', 'desc', 'title', 'last_credit_pull_d', 'next_pymnt_d', 'risk_score'
]

In [35]:
X = df_2.loc[:, df_2.columns != 'target'].drop(drop_list, axis=1)#, 'risk_score'
y = df_2['target']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [36]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import TargetEncoder

# Define transformers for numerical and categorical features
num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('target_encoder', TargetEncoder())
])

# Combine transformers into a column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, selector(dtype_include='number')),
        ('cat', cat_transformer, selector(dtype_include='object'))
    ]
)

# Create the final pipeline
Transf = Pipeline(steps=[
    ('preprocessor', preprocessor)
])

# Fit the pipeline on training data
Transf.fit(X_train, y_train)

/home/athyrson/Code/py10/lib64/python3.10/site-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['member_id' 'mths_since_last_major_derog' 'annual_inc_joint' 'dti_joint'
 'tot_coll_amt' 'tot_cur_bal' 'open_acc_6m' 'open_act_il' 'open_il_12m'
 'open_il_24m' 'mths_since_rcnt_il' 'total_bal_il' 'il_util' 'open_rv_12m'
 'open_rv_24m' 'max_bal_bc' 'all_util' 'total_rev_hi_lim' 'inq_fi'
 'total_cu_tl' 'inq_last_12m' 'acc_open_past_24mths' 'avg_cur_bal'
 'bc_open_to_buy' 'bc_util' 'mo_sin_old_il_acct' 'mo_sin_old_rev_tl_op'
 'mo_sin_rcnt_rev_tl_op' 'mo_sin_rcnt_tl' 'mort_acc'
 'mths_since_recent_bc' 'mths_since_recent_bc_dlq' 'mths_since_recent_inq'
 'mths_since_recent_revol_delinq' 'num_accts_ever_120_pd' 'num_actv_bc_tl'
 'num_actv_rev_tl' 'num_bc_sats' 'num_bc_tl' 'num_il_tl' 'num_op_rev_tl'
 'num_rev_accts' 'num_rev_tl_bal_gt_0' 'num_sats' 'num_tl_120dpd_2m'
 'num_tl_30dpd' 'num_tl_90g_dpd_24m' 'num_tl_op_past_12m' 'pct_tl_nvr_dlq'
 'percen

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fd612ab6560>),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('target_encoder',
                                                                   TargetEncoder())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fd612ab5f00>)]))])

In [37]:
Xt_train = Transf.transform(X_train)

/home/athyrson/Code/py10/lib64/python3.10/site-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['member_id' 'mths_since_last_major_derog' 'annual_inc_joint' 'dti_joint'
 'tot_coll_amt' 'tot_cur_bal' 'open_acc_6m' 'open_act_il' 'open_il_12m'
 'open_il_24m' 'mths_since_rcnt_il' 'total_bal_il' 'il_util' 'open_rv_12m'
 'open_rv_24m' 'max_bal_bc' 'all_util' 'total_rev_hi_lim' 'inq_fi'
 'total_cu_tl' 'inq_last_12m' 'acc_open_past_24mths' 'avg_cur_bal'
 'bc_open_to_buy' 'bc_util' 'mo_sin_old_il_acct' 'mo_sin_old_rev_tl_op'
 'mo_sin_rcnt_rev_tl_op' 'mo_sin_rcnt_tl' 'mort_acc'
 'mths_since_recent_bc' 'mths_since_recent_bc_dlq' 'mths_since_recent_inq'
 'mths_since_recent_revol_delinq' 'num_accts_ever_120_pd' 'num_actv_bc_tl'
 'num_actv_rev_tl' 'num_bc_sats' 'num_bc_tl' 'num_il_tl' 'num_op_rev_tl'
 'num_rev_accts' 'num_rev_tl_bal_gt_0' 'num_sats' 'num_tl_120dpd_2m'
 'num_tl_30dpd' 'num_tl_90g_dpd_24m' 'num_tl_op_past_12m' 'pct_tl_nvr_dlq'
 'percen

In [39]:
Xt_train.shape

(10029, 40)

In [33]:
mutual_info_classif(Xt_train, y_train)

array([0.00169835, 0.00438724, 0.00783498, 0.00648022, 0.02966678,
       0.01259563, 0.        , 0.00871277, 0.        , 0.        ,
       0.01175288, 0.01229849, 0.01345053, 0.00123185, 0.00341657,
       0.00490169, 0.005707  , 0.00073665, 0.00687732, 0.        ,
       0.00163984, 0.00096275, 0.00409681, 0.00294475, 0.00017386,
       0.00480973, 0.        , 0.00512015, 0.00286219, 0.01891568,
       0.02385965, 0.        , 0.        , 0.00685418, 0.02839398,
       0.00540255, 0.00914004, 0.        , 0.00536576, 0.00194431])

In [11]:
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, recall_score, balanced_accuracy_score
import pandas as pd
from sklearn.preprocessing import TargetEncoder
from interpret.glassbox import ExplainableBoostingClassifier  # Import EBM

# Train benchmark models with all features
# Logistic Regression
log_reg = make_pipeline(TargetEncoder(), StandardScaler(), LogisticRegression(max_iter=1000, random_state=42))
log_reg.fit(X_train, y_train)
log_reg_pred = log_reg.predict_proba(X_test)[:, 1]

# Decision Tree Classifier
tree_clf = make_pipeline(TargetEncoder(), StandardScaler(), DecisionTreeClassifier(max_depth=5, random_state=42))
tree_clf.fit(X_train, y_train)
tree_clf_pred = tree_clf.predict_proba(X_test)[:, 1]

# Gradient Boosting Classifier
gb_clf = make_pipeline(TargetEncoder(), StandardScaler(), GradientBoostingClassifier(n_estimators=100, learning_rate=0.01, max_depth=3, random_state=42))
gb_clf.fit(X_train, y_train)
gb_clf_pred = gb_clf.predict_proba(X_test)[:, 1]

# Explainable Boosting Machine (EBM)
ebm = make_pipeline( TargetEncoder(), StandardScaler(), ExplainableBoostingClassifier(random_state=42))
ebm.fit(X_train, y_train)
ebm_pred = ebm.predict_proba(X_test)[:, 1]

# Evaluate performance of benchmark models
log_reg_auc = roc_auc_score(y_test, log_reg_pred)
log_reg_recall = recall_score(y_test, log_reg_pred.round())
log_reg_balanced_acc = balanced_accuracy_score(y_test, log_reg_pred.round())

tree_clf_auc = roc_auc_score(y_test, tree_clf_pred)
tree_clf_recall = recall_score(y_test, tree_clf_pred.round())
tree_clf_balanced_acc = balanced_accuracy_score(y_test, tree_clf_pred.round())

gb_clf_auc = roc_auc_score(y_test, gb_clf_pred)
gb_clf_recall = recall_score(y_test, gb_clf_pred.round())
gb_clf_balanced_acc = balanced_accuracy_score(y_test, gb_clf_pred.round())

ebm_auc = roc_auc_score(y_test, ebm_pred)
ebm_recall = recall_score(y_test, ebm_pred.round())
ebm_balanced_acc = balanced_accuracy_score(y_test, ebm_pred.round())

# Create a DataFrame to store the results
results_df = pd.DataFrame({
    "Model": ["Logistic Regression", "Decision Tree Classifier", "Gradient Boosting Classifier", "Explainable Boosting Machine (EBM)"],
    "AUC": [log_reg_auc, tree_clf_auc, gb_clf_auc, ebm_auc],
    "Recall": [log_reg_recall, tree_clf_recall, gb_clf_recall, ebm_recall],
    "Balanced Accuracy": [log_reg_balanced_acc, tree_clf_balanced_acc, gb_clf_balanced_acc, ebm_balanced_acc]
})

# Display the DataFrame
results_df.round(3)

,Model,AUC,Recall,Balanced Accuracy
0,Logistic Regression,0.678,0.023,0.508
1,Decision Tree Classifier,0.642,0.017,0.504
2,Gradient Boosting Classifier,0.663,0.000,0.500
3,Explainable Boosting Machine (EBM),0.678,0.032,0.511


In [12]:
X = df_2.loc[:, df_2.columns != 'target'].drop(drop_list, axis=1)#, 'risk_score'
y = df_2['target']

# df_3 = create_bunch_df(X, y)
# X = df_3.data
# y = df_3.target
# feature_names = df_3.feature_names

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, recall_score, balanced_accuracy_score
import pandas as pd
from interpret.glassbox import ExplainableBoostingClassifier  # Import EBM

# Train benchmark models with all features



# Explainable Boosting Machine (EBM)
ebm = ExplainableBoostingClassifier(random_state=42)
ebm.fit(X_train, y_train)
ebm_pred = ebm.predict_proba(X_test)[:, 1]

# Evaluate performance of benchmark models
ebm_auc = roc_auc_score(y_test, ebm_pred)
ebm_recall = recall_score(y_test, ebm_pred.round())
ebm_balanced_acc = balanced_accuracy_score(y_test, ebm_pred.round())

# Create a DataFrame to store the results
results_df = pd.DataFrame({
    "Model": ["Logistic Regression", "Decision Tree Classifier", "Gradient Boosting Classifier", "Explainable Boosting Machine (EBM)"],
    "AUC": [log_reg_auc, tree_clf_auc, gb_clf_auc, ebm_auc],
    "Recall": [log_reg_recall, tree_clf_recall, gb_clf_recall, ebm_recall],
    "Balanced Accuracy": [log_reg_balanced_acc, tree_clf_balanced_acc, gb_clf_balanced_acc, ebm_balanced_acc]
})

# Display the DataFrame
results_df.round(3)

/home/athyrson/Code/py10/lib64/python3.10/site-packages/interpret/utils/_preprocessor.py:358: RuntimeWarning: All-NaN slice encountered
  min_feature_val = np.nanmin(X_col)
/home/athyrson/Code/py10/lib64/python3.10/site-packages/interpret/utils/_preprocessor.py:359: RuntimeWarning: All-NaN slice encountered
  max_feature_val = np.nanmax(X_col)
/home/athyrson/Code/py10/lib64/python3.10/site-packages/interpret/glassbox/_ebm/_ebm.py:813: UserWarning: Missing values detected. Our visualizations do not currently display missing values. To retain the glassbox nature of the model you need to either set the missing values to an extreme value like -1000 that will be visible on the graphs, or manually examine the missing value score in ebm.term_scores_[term_index][0]
  warn(


,Model,AUC,Recall,Balanced Accuracy
0,Logistic Regression,0.678,0.023,0.508
1,Decision Tree Classifier,0.642,0.017,0.504
2,Gradient Boosting Classifier,0.663,0.000,0.500
3,Explainable Boosting Machine (EBM),0.697,0.026,0.510


In [14]:
from interpret import show
show(ebm.explain_global())

<!-- http://127.0.0.1:7001/139708102980080/ -->